# Import

In [4]:
import numpy as np
import pandas as pd 

# Data Load

#### 추가적인 Tag 변화

- ##### 1과 1 사이의 0이 있으면 1로 변경해주는 data
- 예시) 

1 0 0 0 0 1 0 0  => 1 1 1 1 1 0 0 변환

In [5]:
# 데이터 불러오기
train_data = pd.read_csv('./Dataset/train_time_data_pf.csv')
val_data = pd.read_csv('./Dataset/validation_time_data_pf.csv')

data = pd.concat([train_data, val_data])
data = data.reset_index(drop = True)

# Data Structure

In [6]:
# [1] 분 단위로 cycle별 Tag 저자 dataFrame 생성
from tqdm import tqdm 

data['STD_DT'] = data['STD_DT'].apply(lambda x : pd.to_datetime(str(x), format='%Y-%m-%d %H:%M:%S'))
minute_result = []
title_list = []
cnt = data.shape[0]

for idx in tqdm(range(data.shape[0])):
    
    # 분 단위로 Dataset 저장하기 위한 코드
    if idx == 0 :                     
        result = []      
    elif idx % 10 == 0 :
        minute_result.append(result)
        result = []
    
    Tag = data['TAG'][idx]
    result.append(Tag)
    
    if idx <= 9:
        name = data.columns[-1]
        second = str(data['STD_DT'].dt.second[idx])
            
        if len(second) == 1:
            second = '0' + second  
            
        change_title = name + '_' + second
        title_list.append(change_title)
    
    if idx == (cnt - 1):
        minute_result.append(result)

100%|██████████████████████████████████████████████████████████████████████| 835200/835200 [00:04<00:00, 182514.05it/s]


In [7]:
## 분 index
df_byminute = data.set_index('STD_DT')['TAG'].resample('T').sum()        
day_list = list(pd.DataFrame(df_byminute).index)

## 분마다의 Tag dataFrame 생성
tag_pd = pd.DataFrame(minute_result)
tag_pd.index = day_list
tag_pd.columns = title_list
print(tag_pd)

                     TAG_00  TAG_06  TAG_12  TAG_18  TAG_24  TAG_30  TAG_36  \
2020-03-04 00:00:00       0       0       0       0       0       0       0   
2020-03-04 00:01:00       0       0       0       0       0       0       0   
2020-03-04 00:02:00       0       0       0       0       0       0       0   
2020-03-04 00:03:00       0       0       0       0       0       0       0   
2020-03-04 00:04:00       0       0       0       0       0       0       0   
...                     ...     ...     ...     ...     ...     ...     ...   
2020-04-30 23:55:00       0       0       0       0       0       0       0   
2020-04-30 23:56:00       0       0       0       0       0       0       0   
2020-04-30 23:57:00       0       0       0       0       0       0       0   
2020-04-30 23:58:00       0       0       0       0       0       0       0   
2020-04-30 23:59:00       0       0       0       0       0       0       0   

                     TAG_42  TAG_48  TAG_54  
2020-

In [ ]:
## Tag 전처리
change_tag = [] 

# for day_num in tqdm(range(tag_pd.shape[0])):

for day_num in tqdm(range(tag_pd.shape[0])):
    
    day_anomaly = tag_pd.iloc[day_num]
    day_anomaly_cnt = day_anomaly.sum()

    one_tag_idx = []
    
    # 1분 주기 동안 Tag가 "NO"인 경우가 1개 이하 
    if day_anomaly_cnt <= 1:
        change_tag.append(list(day_anomaly))
        pass
    
    # 1분 주기 동아 Tag가 2개 이상인 경우
    ## 1분 동안의 1이 등장하는 index를 기준으로 최소값과 최대값을 구하고 그 사이를 0을 ㅗ변환
    else:
        change_day_anomaly = []
        for idx, num in enumerate(day_anomaly):
            if num == 1:
                one_tag_idx.append(idx)
            else:
                pass
            
        max_idx = max(one_tag_idx)
        min_idx = min(one_tag_idx)
        
        # 각 분마다의 초 iindex를 기준으로 0으로 변환
        for idx, num in enumerate(day_anomaly):
            if min_idx <= idx <= max_idx:
                change_day_anomaly.append(1)
            else:
                change_day_anomaly.append(0)
                
        change_tag.append(change_day_anomaly)
        
chagnge_tag_pd = pd.DataFrame(change_tag)
chagnge_tag_pd.index = day_list
chagnge_tag_pd.columns = title_list
print(chagnge_tag_pd)

 45%|█████████████████████████████████▏                                        | 37460/83520 [00:04<00:05, 8557.10it/s]

In [ ]:
# 잘바뀌었는지 검증 time
ver = df_byminute.reset_index()

# No day
No_day = list(ver[ver['TAG'] != 0]['STD_DT'])
No_idx = []

for day in tqdm(No_day):
    value_idx = ver[ver['STD_DT'] == day].index[0]
    No_idx.append(value_idx)

import random
random.seed(0)

random_idx = random.sample(No_idx, 20)

# 변경 이상있는지 Sampling을 통해서 확인
for idx in random_idx:
    bef_tag = pd.DataFrame(tag_pd.iloc[idx]).T
    aft_tag = pd.DataFrame(chagnge_tag_pd.iloc[idx]).T
    
    print("TAG가 NO 개수: {}개".format(sum(tag_pd.iloc[idx])))
    print("변경 전:")
    display(bef_tag)
    print("변경 전:")
    display(aft_tag)
    print("\n")

In [197]:
# Tag 변경 코드
total_list = []

for c_idx in tqdm(range(chagnge_tag_pd.shape[0])):
    minute_value = chagnge_tag_pd.iloc[c_idx]
    for per_value in minute_value:
        total_list.append(per_value)
        
change_dt = data.copy()
change_dt['TAG'] = total_list

print("이전 TAG 개수 \n {}".format(data['TAG'].value_counts()))
print("이후 TAG 개수 \n {}".format(change_dt['TAG'].value_counts()))

100%|█████████████████████████████████████████████████████████████████████████| 83520/83520 [00:05<00:00, 16105.30it/s]


이전 TAG 개수 
 0    658133
1    177067
Name: TAG, dtype: int64
이후 TAG 개수 
 0    519391
1    315809
Name: TAG, dtype: int64


In [202]:
# 값 저장 
train = change_dt[change_dt['STD_DT'] < '2020-04-08']
test = change_dt[change_dt['STD_DT'] >= '2020-04-08']

train.to_csv('train_result_df.csv')
test.to_csv('test_result_df.csv')